backpropgation algorithm

In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
uploaded = files.upload()
npzfile = np.load('CBCL.npz')
trainface = npzfile['arr_0']
trainnonface = npzfile['arr_1']
testface = npzfile['arr_2']
testnonface = npzfile['arr_3']
print(trainface.shape) #(2429, 361)
print(trainnonface.shape) #(4548, 361)
print(testface.shape) #(472, 361)
print(testnonface.shape) #(23573, 361)

Saving CBCL.npz to CBCL.npz
(2429, 361)
(4548, 361)
(472, 361)
(23573, 361)


In [ ]:
def BPNNtrain(pf,nf,hn,lr,epoch):
  pn = pf.shape[0] #2429
  nn = nf.shape[0] #4548
  fn = pf.shape[1] #361
  feature = np.append(pf,nf,axis=0)
  target = np.append(np.ones((pn,1)),np.zeros((nn,1)),axis=0)
  WI = np.random.normal(0,1,(fn+1,hn))
  WO = np.random.normal(0,1,(hn+1,1))
  for t in range(epoch):
    s = random.sample(range(pn+nn),pn+nn)
    for i in range(pn+nn):
      ins = np.append(feature[s[i],:],1)
      ho = ins.dot(WI)
      ho = 1/(1+np.exp(-ho))
      hs = np.append(ho,1)
      out = hs.dot(WO)
      out = 1/(1+math.exp(-out))
      dk = out*(1-out)*(target[s[i]]-out)
      dh = ho*(1-ho)*WO[:hn,0]*dk
      WO[:,0] += lr*dk*hs
      for j in range(hn):
        WI[:,j] += lr*dh[j]*ins
  model = dict()
  model['WI'] = WI
  model['WO'] = WO
  return model

def BPNNtest(feature,model):
  sn = feature.shape[0]
  WI = model['WI']
  WO = model['WO']
  hn = WI.shape[1]
  out = np.zeros((sn,1))
  for i in range(sn):
    ins = np.append(feature[i,:],1)
    ho = ins.dot(WI)
    ho = 1/(1+np.exp(-ho))
    hs = np.append(ho,1)
    out[i] = hs.dot(WO)
    out[i] = 1/(1+math.exp(-out[i]))
  return out

network = BPNNtrain(trainface/255,trainnonface/255,20,0.01,10)

/tmp/ipython-input-1427850540.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  out = 1/(1+math.exp(-out))


In [ ]:
# training ROC curve
pscore = BPNNtest(trainface/255,network)
nscore = BPNNtest(trainnonface/255,network)
x = np.zeros((1001,))
y = np.zeros((1001,))
for i in range(1001):
  threshold = i/1000
  y[i] = np.sum(pscore>=threshold)/pscore.shape[0]
  x[i] = np.sum(nscore>=threshold)/nscore.shape[0]
plt.plot(x,y)
plt.show()

In [ ]:
# testing ROC curve
pscore = BPNNtest(testface/255,network)
nscore = BPNNtest(testnonface/255,network)
x = np.zeros((1001,))
y = np.zeros((1001,))
for i in range(1001):
  threshold = i/1000
  y[i] = np.sum(pscore>=threshold)/pscore.shape[0]
  x[i] = np.sum(nscore>=threshold)/nscore.shape[0]
plt.plot(x,y)
plt.show()